In [ ]:
#  ranodm splite_predict, section 1: random split  data in to train and te for each year. 
# secction2  tune parameter
# section 3 all year rmse and rsq plot, varible importance plot
# section 4 single year yy plot
#section 5 partial depence plot

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import scipy
import math

from collections import Counter
from sklearn import tree


from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestRegressor

from statistics import variance


from sklearn.datasets import make_regression
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.inspection import partial_dependence
from sklearn.inspection import PartialDependenceDisplay
import statistics as st


from numpy.random import default_rng

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import scale



import random


seed_no=111
random.seed(seed_no)
np.random.seed(seed_no)
plt.rcParams.update({
#     "font.weight": "bold",
    "figure.figsize":"8, 5",
    "font.size":"15",
#     "grid.color": "0.5",
#     "grid.linestyle": "-",
#     "grid.linewidth": 5,
#     "lines.linewidth": 2,
#     "lines.color": "g",
})
def rsq(y,yh):
    y_mean=np.mean(y)
    n=len(y)
#     tss_n=np.sum((y-y_mean)**2)/n
    tss=np.sum((y-y_mean)**2)
    rss=np.sum((y-yh)**2)
    return (1-rss/tss, tss/n,rss)
# def rsq(y,yh):
#     y_mean=np.mean(y)
#     tss=np.sum((y-y_mean)**2)
#     rss=np.sum((y-yh)**2)
#     return (1-rss/tss)
def rmse(y,yh):
    return(math.sqrt(np.mean((y-yh)**2)))
def bias(y,yh):
    return (sum(y-yh)/y)
def pbias(y_sim,y_obs):
    a=np.sum(y_sim-y_obs)
    b=np.sum(y_obs)
    return (100*a/b)

# Section 1, prepare random split dataset


In [2]:
'''steps: 
1: find the datapoint that appear in every year
2: random split these data in 70% train 30% test
'''
variables=['denies_area','sb','sgh','ww','corn','alff','vpd','tem','et','pr','gimerged','ndvimerged','ndwimerged'] 
cd=pd.read_csv('D:/all_GEE/aggregate620_merged_NORM.csv')# read file
year= np.arange(6,21) 
dat1 = cd[cd.irri!= 0]
dat3 = dat1[dat1.denies_area!= 0]
dat2=dat3.dropna()
# percentage_variables=['sb','sgh','ww','corn','alff']
# dat2[percentage_variables]=dat2[percentage_variables]*900/36000000
# dat2['et']=dat2['et']*500*500/36000000
# dat2['irri']=dat2['irri']*1233.48*1000/36000000
# dat2['pr']=dat2['pr']*16000000/36000000

In [3]:
lb=dat2['label'].to_numpy()
label = Counter(lb) # counter each lable appearance 
# print(label)
duplicate_list = list([item for item in label if label[item]>len(year)-1]) # keep lable that  appear every year (total 12 years, large than 11 times)
unique_list=list([item for item in label if label[item]<len(year)])# filter lable that not appear every year ( less than 12 times)
mask_dup=np.in1d(lb,duplicate_list) # mask the dupicate lable 
mask_unique=np.in1d(lb,unique_list)# mask the unique lable
dat_dup=dat2[mask_dup]
dat_uni=dat2[mask_unique]
print(dat_dup.shape)
print(dat2.shape)
print(dat_uni.shape)

(26115, 17)
(29045, 17)
(2930, 17)


In [4]:
appended_data_tr = []
appended_data_te = []
i_label_rnd_tr=random.sample(range(0, len(duplicate_list )), math.floor(0.7*len(duplicate_list ))) # random split duplicate label in 70%
i_label_rnd_te= [x for x in range(len(duplicate_list )) if x not in i_label_rnd_tr] # the rest 30 % is test 
print(len(i_label_rnd_tr))
print(len(i_label_rnd_te))
for i in range (len(year)): # loop over each year and aggreaget the same 70% to train, 30% to test
    dat_sy=dat_dup[(dat_dup.year== year[i])]
    dat_tr=dat_sy.iloc[i_label_rnd_tr, :]
    dat_te= dat_sy.iloc[i_label_rnd_te, :]
    appended_data_tr.append(dat_tr)
    appended_data_te.append(dat_te)
appended_data_tr = pd.concat(appended_data_tr)   
appended_data_te = pd.concat(appended_data_te) 
print(appended_data_tr.shape)
print(appended_data_te.shape)
# appended_data_te.to_excel('C:/Users/swei46.ASURITE/Desktop/GEE6km6-17/appended_te.xlsx')
# appended_data_tr.to_excel('C:/Users/swei46.ASURITE/Desktop/GEE6km6-17/appended_tr.xlsx')

1218
523
(18270, 17)
(7845, 17)


# Section 2 tune parameter 

In [ ]:
'''cross validation of 2006 to 2017 data,  with each year of data_te as test dat and the training data is from dat_tr of  rest of year. '''
year=np.arange(6,21)
print(year)
n_minleafesize=np.array([1,10,20,40,60,80,100])
n_estimator=np.arange(3,13)
bicrsq_te=np.zeros((len(n_minleafesize),len(n_estimator)))
bicrsq_tr=np.zeros((len(n_minleafesize),len(n_estimator)))
bicrmse_te=np.zeros((len(n_minleafesize),len(n_estimator)))
bicrmse_tr=np.zeros((len(n_minleafesize),len(n_estimator)))
rsq_te=np.zeros((len(year),1))
rsq_tr=np.zeros ((len(year),1))
rmse_te=np.zeros ((len(year),1))
rmse_tr=np.zeros((len(year),1))
print(rmse_tr[5])
print(bicrmse_tr.shape)

In [ ]:
'"random split"'


for i in range(len(n_minleafesize)):
    for k in range(len(n_estimator)):
        for u in range(len(year)):
            dat_te=appended_data_te[(appended_data_te.year== year[u])]
            dat_tr=appended_data_tr#[(appended_data_tr.year!= year[u]) ]
            X_train=dat_tr[variables].to_numpy()
            y_train =np.ravel(dat_tr[['irri']]. to_numpy())
            X_test=dat_te[variables].to_numpy()
            y_test =np.ravel(dat_te[['irri']]. to_numpy())
            rfm = RandomForestRegressor(n_jobs=-1,n_estimators=800,random_state=seed_no,max_features=n_estimator[k],min_samples_leaf=n_minleafesize[i])
            rfm.fit(X_train,y_train)
            yhte= rfm.predict(X_test)
            yhtr=rfm.predict(X_train)
            rsq_te[u]=rsq(y_test,yhte)# rsq per year for a single feature parameter and leaf 
            rsq_tr[u]=rsq(y_train,yhtr)      
            rmse_te[u]=rmse(y_test,yhte)
            rmse_tr[u]=rmse(y_train,yhtr)
            
        bicrsq_te[i,k]=np.mean(rsq_te)
        bicrsq_tr[i,k]=np.mean(rsq_tr)
        bicrmse_te[i,k]=np.mean(rmse_te)
        bicrmse_tr[i,k]=np.mean(rmse_tr)

In [ ]:
'"regional splite"'
for i in range(len(n_minleafesize)):
    for k in range(len(n_estimator)):
        for u in range(len(year)):
            dat_test=dat2[(dat2.label>2665)& (dat2.year==year[u])]
            dat_train=dat2[(dat2.label<2665)]
            X_test=dat_test[variables].to_numpy()
            X_train=dat_train[variables].to_numpy()
            y_test=np.ravel(dat_test['irri'].to_numpy())
            y_train=np.ravel(dat_train['irri'].to_numpy())
            rfm = RandomForestRegressor(n_jobs=-1,n_estimators=800,random_state=seed_no,max_features=n_estimator[k],min_samples_leaf=n_minleafesize[i])
            rfm.fit(X_train,y_train)
            ypred_test=rfm.predict(X_test)
            ypred_train=rfm.predict(X_train)
            rsq_te[u]=rsq(y_test,ypred_test)# rsq per year for a single feature parameter and leaf 
            rsq_tr[u]=rsq(y_train,ypred_train)      
            rmse_te[u]=rmse(y_test,ypred_test)
            rmse_tr[u]=rmse(y_train,ypred_train)
            
        bicrsq_te[i,k]=np.mean(rsq_te)
        bicrsq_tr[i,k]=np.mean(rsq_tr)
        bicrmse_te[i,k]=np.mean(rmse_te)
        bicrmse_tr[i,k]=np.mean(rmse_tr)

In [ ]:
fig,ax=plt.subplots(2)
plt.legend()
plt.xlabel('number of estimators',fontsize=15)
# plt.title('test performance of RF model tuning parameter for all-year( 2006 -2017) random split',fontsize=15)
for i in range(len(n_minleafesize)):
    ax[0].plot(n_estimator, bicrsq_te[i,:],label=n_minleafesize[i])    
    ax[1].plot(n_estimator,bicrmse_te[i,:],label=n_minleafesize[i])
    ax[0].set_ylabel('R square')
    ax[1].set_ylabel('RMSE (Acre-Feet)')
#     )
#     plt.ylabel('R square',fontsize=15)
#     plt.title('training performance of RF model tuning parameter for all-year( 2006 -2017) random split',fontsize=15)
plt.legend()
# plt.savefig('C:/Users/swei46.ASURITE/OneDrive/Documents/weekly_repo/manuscript/manuscript_fig/regional_split_tune_testparameters',bbox_inches='tight',dpi=500)

In [ ]:
fig,ax=plt.subplots(2)
plt.legend()
plt.xlabel('number of estimators',fontsize=15)
# plt.title('test performance of RF model tuning parameter for all-year( 2006 -2017) random split',fontsize=15)
for i in range(len(n_minleafesize)):
    ax[0].plot(n_estimator, bicrsq_tr[i,:],label=n_minleafesize[i])    
    ax[1].plot(n_estimator,bicrmse_tr[i,:],label=n_minleafesize[i])
    ax[0].set_ylabel('R square')
    ax[1].set_ylabel('RMSE (Acre-Feet)')
#     )
#     plt.ylabel('R square',fontsize=15)
#     plt.title('training performance of RF model tuning parameter for all-year( 2006 -2017) random split',fontsize=15)
plt.legend()
# plt.savefig('C:/Users/swei46.ASURITE/OneDrive/Documents/weekly_repo/manuscript/manuscript_fig/regional_split_tune_parameters_trainplot',bbox_inches='tight',dpi=500)
#best parameter: number of estimator: 3; leaf size:10

# Section 3 all year rmse and yy plot

In [5]:
'"random split  Ex A.1"'
y_sum=np.zeros((len(year),2))
rsq_t=np.zeros ((len(year),2))
tss_t=np.zeros ((len(year),2))
rss_t=np.zeros ((len(year),2))
rmse_t=np.zeros ((len(year),2))
vari_imp_mn=np.zeros((len(year),len(variables)))
pr_timeseries=np.zeros((len(year),2))
irri_intensity=np.zeros((len(year),2))
pbias_t=np.zeros ((len(year),1))
for u in range(len(year)):
    dat_te=appended_data_te[(appended_data_te.year== year[u])]
    dat_tr=appended_data_tr#[(appended_data_tr.year!= year[u]) ]
#     dat_tr=appended_data_tr
    X_train=dat_tr[variables].to_numpy()
    y_train =np.ravel(dat_tr[['irri']]. to_numpy())
    X_test=dat_te[variables].to_numpy()
    y_test =np.ravel(dat_te[['irri']]. to_numpy())
    rfm = RandomForestRegressor(n_jobs=-1,n_estimators=800,random_state=seed_no,max_features=13,min_samples_leaf=20)
    rfm.fit(X_train,y_train)
    yhte= rfm.predict(X_test)
    yhtr=rfm.predict(X_train)
    y_sum[u,0]=np.sum(yhte)
    y_sum[u,1]=np.sum(y_test)
    rsq_t[u,0],tss_t[u,0], rss_t[u,0]=rsq(y_test,yhte)
    rsq_t[u,1],tss_t[u,1], rss_t[u,1]=rsq(y_train,yhtr)
    pbias_t[u]=pbias(yhte,y_test)
    rmse_t[u,0]=rmse(y_test,yhte)
    rmse_t[u,1]=rmse(y_train,yhtr)
    pr_timeseries[u,1]=np.mean(dat_tr['pr'].to_numpy())
    pr_timeseries[u,0]=np.mean(dat_te['pr'].to_numpy())
    vari_imp_mn[u,:]=rfm.feature_importances_

In [ ]:
 pbias_t.mean(),np.sqrt(tss_t[:,0])
plt.hist(y_test)

In [ ]:
from sklearn.metrics import mean_squared_error

rms = mean_squared_error(y_test, yhte, squared=False)
rms

In [ ]:
from sklearn.metrics import r2_score
r2_score(y_test, yhte)

In [ ]:

plt.plot(year,tss_t[:,0],label='Variance',  marker='^', color='g', alpha=0.5)  
plt.set_xlabel('Year')
ax2.set_ylabel(r'$R^2$')
ax2.tick_params(axis='y')
ax2.legend(loc='upper right')

In [ ]:
'"regional splite each individual year. train: per year west; test : per year east"'
y_sum=np.zeros((len(year),2))
rsq_t=np.zeros ((len(year),2))
rmse_t=np.zeros ((len(year),2))
vari_imp_mn=np.zeros((len(year),len(variables)))
pr_timeseries=np.zeros((len(year),2))
for u in range(len(year)):
    dat_te=dat2[(dat2.label>2668)&(dat2.year== year[u])]
    dat_tr=dat2[(dat2.label<2668)&(dat2.year== year[u])]
    dat_tr_year=dat2[(dat2.label<2668)&(dat2.year== year[u])]
    X_train=dat_tr[variables].to_numpy()
    y_train =np.ravel(dat_tr[['irri']]. to_numpy())
    X_test=dat_te[variables].to_numpy()
    y_test =np.ravel(dat_te[['irri']]. to_numpy())
    rfm = RandomForestRegressor(n_jobs=-1,n_estimators=800,random_state=seed_no,max_features=13,min_samples_leaf=100)
    rfm.fit(X_train,y_train)
    yhte= rfm.predict(X_test)
    yhtr=rfm.predict(X_train)
    y_sum[u,0]=np.sum(yhte)
    y_sum[u,1]=np.sum(y_test)
    rsq_t[u,0]=rsq(y_test,yhte)
    rsq_t[u,1]=rsq(y_train,yhtr)
    rmse_t[u,0]=rmse(y_test,yhte)
    rmse_t[u,1]=rmse(y_train,yhtr)
    pr_timeseries[u,1]=np.mean(dat_tr_year['pr'].to_numpy())
    pr_timeseries[u,0]=np.mean(dat_te['pr'].to_numpy())
    vari_imp_mn[u,:]=rfm.feature_importances_

In [3]:
'"regional split without temporal  Ex B"'"'
v_n=['pr']
year=np.arange(6,21)
y_sum=np.zeros((len(year),2))
rsq_t=np.zeros ((len(year),2))
rmse_t=np.zeros ((len(year),2))
vari_imp_mn=np.zeros((len(year),len(variables)))
irri_intensity=np.zeros((len(year),2))
pr_timeseries=np.zeros((len(year),3))
for i in range(len(year)):
    dat_te=dat2[(dat2.label>2668)&(dat2.year==year[i]) ]
    dat_tr_year=dat2[(dat2.label<2668)&(dat2.year==year[i]) ]
    dat_tr=dat2[(dat2.label<2668)]
    X_train=dat_tr[variables].to_numpy()
    y_train =np.ravel(dat_tr[['irri']]. to_numpy())
    X_test=dat_te[variables].to_numpy()
    y_test =np.ravel(dat_te[['irri']]. to_numpy())
    rfm = RandomForestRegressor(n_jobs=-1,n_estimators=800,random_state=seed_no,max_features=3,min_samples_leaf=10)
    rfm.fit(X_train,y_train)
    yhte= rfm.predict(X_test)
    yhtr=rfm.predict(X_train)
    y_sum[i,0]=np.mean(yhte)
    y_sum[i,1]=np.mean(y_test)
    rsq_t[i,0]=rsq(y_test,yhte)
    rsq_t[i,1]=rsq(y_train,yhtr)
    rmse_t[i,0]=rmse(y_test,yhte)
    rmse_t[i,1]=rmse(y_train,yhtr)
    pr_timeseries[i,1]=np.mean(dat_tr[v_n].to_numpy())
    pr_timeseries[i,0]=np.mean(dat_te[v_n].to_numpy())
    pr_timeseries[i,2]=np.mean(dat_tr_year[v_n].to_numpy())
    irri_intensity[i,1]=np.mean(dat_tr['irri']/ dat_tr['denies_area'].to_numpy())
    irri_intensity[i,0]=np.mean( y_test/dat_te['denies_area'].to_numpy())
    vari_imp_mn[i,:]=rfm.feature_importances_

In [ ]:
'"regional split without temporal train: east, test : west"'
v_n=['pr']
year=np.arange(6,21)
y_sum=np.zeros((len(year),2))
rsq_t=np.zeros ((len(year),2))
rmse_t=np.zeros ((len(year),2))
vari_imp_mn=np.zeros((len(year),len(variables)))
irri_intensity=np.zeros((len(year),2))
pr_timeseries=np.zeros((len(year),3))
for i in range(len(year)):
    dat_te=dat2[(dat2.label<2668)&(dat2.year==year[i]) ]
    dat_tr_year=dat2[(dat2.label>2668)&(dat2.year==year[i]) ]
    dat_tr=dat2[(dat2.label>2668)]
    X_train=dat_tr[variables].to_numpy()
    y_train =np.ravel(dat_tr[['irri']]. to_numpy())
    X_test=dat_te[variables].to_numpy()
    y_test =np.ravel(dat_te[['irri']]. to_numpy())
    rfm = RandomForestRegressor(n_jobs=-1,n_estimators=800,random_state=seed_no,max_features=6,min_samples_leaf=10)
    rfm.fit(X_train,y_train)
    yhte= rfm.predict(X_test)
    yhtr=rfm.predict(X_train)
    y_sum[i,0]=np.sum(yhte)
    y_sum[i,1]=np.sum(y_test)
    rsq_t[i,0]=rsq(y_test,yhte)
    rsq_t[i,1]=rsq(y_train,yhtr)
    rmse_t[i,0]=rmse(y_test,yhte)
    rmse_t[i,1]=rmse(y_train,yhtr)
    pr_timeseries[i,1]=np.mean(dat_tr[v_n].to_numpy())
    pr_timeseries[i,0]=np.mean(dat_te[v_n].to_numpy())
    pr_timeseries[i,2]=np.mean(dat_tr_year[v_n].to_numpy())
    irri_intensity[i,1]=np.mean(dat_tr['irri']/ dat_tr['area'].to_numpy())
    irri_intensity[i,0]=np.mean( y_test/dat_te['area'].to_numpy())
    vari_imp_mn[i,:]=rfm.feature_importances_

In [3]:
'"temporal Ex A.2"'
year=np.arange(6,21)
y_sum=np.zeros((len(year),2))
area_sum=np.zeros((len(year),1))
rsq_t=np.zeros ((len(year),2))
tss_t=np.zeros ((len(year),2))
rss_t=np.zeros ((len(year),2))
rmse_t=np.zeros ((len(year),2))
pr_timeseries=np.zeros((len(year),2))
vari_imp_mn=np.zeros((len(year),len(variables)))
pbias_t=np.zeros ((len(year),1))
for i in range(len(year)):
    dat_te=dat2[(dat2.year==year[i]) ]
    dat_tr=dat2[(dat2.year!=year[i])]
    X_train=dat_tr[variables].to_numpy()
    y_train =np.ravel(dat_tr[['irri']]. to_numpy())
    X_test=dat_te[variables].to_numpy()
    y_test =np.ravel(dat_te[['irri']]. to_numpy())
    rfm = RandomForestRegressor(n_jobs=-1,n_estimators=800,random_state=seed_no,max_features=5,min_samples_leaf=25)
    rfm.fit(X_train,y_train)
    yhte= rfm.predict(X_test)
    yhtr=rfm.predict(X_train)
    y_sum[i,0]=np.mean(yhte)
    area_sum[i]=np.mean(dat_te['denies_area'])
    y_sum[i,1]=np.mean(y_test)
    rsq_t[i,0],tss_t[i,0], rss_t[i,0]=rsq(y_test,yhte)
    rsq_t[i,1],tss_t[i,1], rss_t[i,1]=rsq(y_train,yhtr)
    rmse_t[i,0]=rmse(y_test,yhte)
    rmse_t[i,1]=rmse(y_train,yhtr)
    pbias_t[i]=pbias(yhte,y_test)
    vari_imp_mn[i,:]=rfm.feature_importances_
    pr_timeseries[i,1]=np.mean(dat_tr['pr'].to_numpy())
    pr_timeseries[i,0]=np.mean(dat_te['pr'].to_numpy())


# plot 

In [ ]:
"'random varaince figure s2"


import statsmodels.api as sm
X2 = sm.add_constant(year)
mod = sm.OLS(tss_t[:,0],X2)
fii = mod.fit()
print(fii.summary())


year=np.arange(2006,2021)
fig, ax2=plt.subplots(dpi=500)


# trend line
z_md=np.polyfit(year, tss_t[:,0],1)
z_fct=np.poly1d(z_md)
tr_line=z_fct(year)
ax2.plot(year,tr_line,label='Trend line of variance',linestyle='--' , color='b', alpha=0.5)
# plt.text(0.6, 0.67, r'$y=-5.076e+07 x +1.035e+11 $ ',  transform=ax2.transAxes)
plt.text(0.6, 0.4, r'$ p = 0.03$ ',  transform=ax2.transAxes,color='blue')

z_fct


ax2.plot(year,tss_t[:,0],label='Variance',  marker='^', color='g', alpha=0.5)         
ax2.set_xlabel('Year')
ax2.set_ylabel('Variance (mm'+ r'$^2$' +')')
ax2.tick_params(axis='y')
ax2.legend(loc='upper right') 
ax2.xaxis.set_ticks_position('both')
ax2.tick_params(direction='in')
# ax2.set_ylim(0, 1)
print(z_fct)
plt.show()
plt.savefig('C:/Users/swei46.ASURITE/OneDrive/Documents/weekly_repo/manuscript/manuscript_fig/figure_fin_v/figures2s _variance_mm.pdf',bbox_inches='tight',dpi=500)
plt.savefig('C:/Users/swei46.ASURITE/OneDrive/Documents/weekly_repo/manuscript/manuscript_fig/figure_fin_v/figures2s _variance_mm.png',bbox_inches='tight',dpi=500)

In [ ]:
rmse_t[:,0].mean()

In [ ]:
"'random split Rsq and Rmse figure 2'"
year=np.arange(2006,2021)
fig, ax1=plt.subplots(dpi=500)
space=0
# ax1.bar(year, rmse_t[:,0]*1233.48/36000000, label='RMSE',color='k',alpha=0.3)   
ax1.bar(year, rmse_t[:,0], label='RMSE',color='k',alpha=0.3)   

ax1.set_xlabel('Year',fontsize=15)
ax1.set_ylabel('RMSE (mm)',fontsize=15)
ax1.tick_params(axis='y')
ax1.legend(loc='upper left', fontsize=12) 
ax1.xaxis.set_ticks_position('both')
ax1.tick_params(direction='in')
# ax1.set_ylim(0, 0.1)
               
ax2 = ax1.twinx()


# trend line
z_md=np.polyfit(year, rsq_t[:,0],1)
z_fct=np.poly1d(z_md)
tr_line=z_fct(year)
ax2.plot(year,tr_line,label=r'Trend line of $R^2$',linestyle='--' , color='b', alpha=0.5)
# plt.text(0.6, 0.67, r'$y=-0.01 x + 21.9$ ',  transform=ax2.transAxes)
plt.text(0.6, 0.67, r'$ p = 0.001$ ',  transform=ax2.transAxes,color='blue', fontsize=12)

z_fct


ax2.plot(year,rsq_t[:,0],label=r'$R^2$',  marker='^', color='g', alpha=0.5)           
ax2.set_xlabel('Year')
ax2.set_ylabel(r'$R^2$')
ax2.tick_params(axis='y')
ax2.legend(loc='upper right',fontsize=12) 
ax2.xaxis.set_ticks_position('both')
ax2.tick_params(direction='in')
ax2.set_ylim(0, 1)

import statsmodels.api as sm
X2 = sm.add_constant(year)
mod = sm.OLS(rsq_t[:,0],X2)
fii = mod.fit()
print(fii.summary())
print(fii.pvalues)
print(z_fct)


fig.savefig('C:/Users/swei46.ASURITE/OneDrive/Documents/weekly_repo/manuscript/manuscript_fig/figure_fin_v/figure2_a _random_rsq_rmse_mm.tif',bbox_inches='tight',dpi=500)
fig.savefig('C:/Users/swei46.ASURITE/OneDrive/Documents/weekly_repo/manuscript/manuscript_fig/figure_fin_v/figure2_a _random_rsq_rmse_mm.png',bbox_inches='tight',dpi=500)
fig.savefig('C:/Users/swei46.ASURITE/OneDrive/Documents/weekly_repo/manuscript/manuscript_fig/figure_fin_v/figure2_a_random_rsq_rmse_mm.pdf',bbox_inches='tight',dpi=500)

plt.show()

In [ ]:
yr=np.arange(2006,2021)
import statsmodels.api as sm
X2 = sm.add_constant(pr_timeseries[:,0])
mod_rf = sm.OLS(rsq_t[:,0],X2)
fi_rf= mod_rf.fit()
fi_rf.summary()


In [ ]:
"'temporal Rsq vs pr,figure 1s'"

fig, ax1 = plt.subplots(dpi=500)
sapce=0.0
# color = 'tab:red'
ax1.set_xlabel('Average growing season (MJJA) precipitation (mm)')
ax1.set_ylabel(r'$R^2$')
ax1.scatter(pr_timeseries[:,0], rsq_t[:,0],marker='*', color='b',alpha=0.4)
ax1.tick_params(axis='y')

ax1.set_ylim(0, 1)

ax1.tick_params(direction='in')

ax1.legend( loc='upper left')

ax1.xaxis.set_ticks_position('both')

# fut trend line
z_md=np.polyfit(pr_timeseries[:,0], rsq_t[:,0],1)
z_fct=np.poly1d(z_md)
tr_line=z_fct(pr_timeseries[:,0])
ax1.plot(pr_timeseries[:,0],tr_line,label=r'Trend line of $R^2$',linestyle='--')
ax1.legend(loc='upper left')

# plt.text(170, 0.9, r'$y=-0.0005 x + 0.95$ ')
plt.text(350, 0.9, r'$p = 0.02$ ')

fig.tight_layout()  # otherwise the right y-label is slightly clipped

plt.savefig('C:/Users/swei46.ASURITE/OneDrive/Documents/weekly_repo/manuscript/manuscript_fig/figure_fin_v/figureS1 _Rsq_vs_pr_temporal.pdf',bbox_inches='tight',dpi=500)
plt.savefig('C:/Users/swei46.ASURITE/OneDrive/Documents/weekly_repo/manuscript/manuscript_fig/figure_fin_v/figureS1 _Rsq_vs_pr_temporal.png',bbox_inches='tight',dpi=500)
plt.show()
print(z_fct)

In [ ]:
"'temporal annual sum figure 3'"





yr=np.arange(2006,2021)
import statsmodels.api as sm
X2 = sm.add_constant(yr)
mod_rf = sm.OLS(y_sum[:,1],X2)
fi_rf= mod_rf.fit()
fi_rf.summary()



# plt.figure(dpi=500)
fig, ax1 = plt.subplots(dpi=500)
sapce=0.2
# color = 'tab:red'
ax1.set_xlabel('Year')
ax1.set_ylabel('Average growing season (MJJA) precipitation (mm)')
# ax1.bar(yr-sapce, pr_timeseries[:,1],0.4, label='Pr_train',color='k',alpha=0.3)
ax1.bar(yr+sapce, pr_timeseries[:,0], 0.4, label='Pr(test)',color='b',alpha=0.3)
ax1.tick_params(axis='y')






ax2 = ax1.twinx()  # instantiate a second axes that shares the same x-axis


ax2.set_ylabel('Spatial average annual irrigation depth (mm)') 
ax2.plot(yr+sapce/2, y_sum[:,0], label='RFR (mm)',marker='^', color='red',linewidth=1, markersize=6,alpha=0.5)#
ax2.plot(yr+sapce/2,y_sum[:,1], label='In situ (mm)',marker='s', color='green',linewidth=1, markersize=6,alpha=0.5)#
  
# trend line insitu



# ax2.plot(yr+sapce/2,-1.07*yr+2219,label='Trend line of in situ',color='green',linestyle ='--')
# # plt.text(0.45, 0.75, r'$y=-5.878e+04*x+1.213e+08$ ',  transform=ax2.transAxes,color="g")
plt.text(0.35, 0.6, r'$ p = 0.022$ ',  transform=ax2.transAxes,color="g")


# trend line
z_md=np.polyfit(yr, y_sum[:,1],1)
z_fct=np.poly1d(z_md)
tr_line=z_fct(yr)
ax2.plot(yr+sapce/2,tr_line,label=r'Trend line of in situ',linestyle='--' , color='green', alpha=0.5)
# plt.text(0.6, 0.67, r'$y=-0.01 x + 21.9$ ',  transform=ax2.transAxes)
# plt.text(0.6, 0.67, r'$ p = 0.001$ ',  transform=ax2.transAxes,color='blue')





ax2.tick_params(axis='y')
ax1.set_xlim(2005, 2021)


ax1.set_ylim(0, 600)

ax1.tick_params(direction='in')
ax2.tick_params(direction='in')
ax1.legend( loc='upper left')
ax2.legend( loc='upper right')
ax1.xaxis.set_ticks_position('both')




ax2.set_ylim(0, 100)

fig.tight_layout()  # otherwise the right y-label is slightly clipped

plt.savefig('C:/Users/swei46.ASURITE/OneDrive/Documents/weekly_repo/manuscript/manuscript_fig/figure_fin_v/figure3 _irri_mean_mm_temporal_620.pdf',bbox_inches='tight',dpi=500)
plt.savefig('C:/Users/swei46.ASURITE/OneDrive/Documents/weekly_repo/manuscript/manuscript_fig/figure_fin_v/figure3 _irri_mean_mm_temporal_620.png',bbox_inches='tight',dpi=500)
plt.savefig('C:/Users/swei46.ASURITE/OneDrive/Documents/weekly_repo/manuscript/manuscript_fig/figure_fin_v/figure3 _irri_mean_mm_temporal_620.tif',bbox_inches='tight',dpi=500)

# plt.show()


In [ ]:
fi_rf.summary(),z_fct

In [ ]:
"'regional y sum figure 4'"
yr=np.arange(2006,2021)
# plt.figure(dpi=500)
fig, ax1 = plt.subplots(dpi=500)
sapce=0.2
# color = 'tab:red'
ax1.set_xlabel('Year')
ax1.set_ylabel('Average growing season (MJJA) precipitation (mm)')
# ax1.bar(yr-sapce, pr_timeseries[:,0],0.4, label='Pr_train',color='k',alpha=0.3)

ax1.tick_params(axis='y')
# ax1.bar(yr-sapce, irri_intensity[:,0], 0.4, label='Pr_test',color='b',alpha=0.3)
# ax1.bar(yr+sapce, irri_intensity[:,1], 0.4, label='Pr_train',color='k',alpha=0.3)
ax1.bar(yr-sapce, pr_timeseries[:,0], 0.4, label='Pr (test)',color='b',alpha=0.3)
ax1.bar(yr+sapce, pr_timeseries[:,2], 0.4, label='Pr (train)',color='k',alpha=0.3)

ax2 = ax1.twinx()  # instantiate a second axes that shares the same x-axis

# color = 'tab:blue'
# ax2.set_ylabel(r'$R^2$') 
# ax2.set_ylabel('Kansas HPA annual irrigation amount (Acre-Feet)') 
ax2.set_ylabel('East region spatial average annual irrigation depth (mm)') 
ax2.plot(yr+sapce/2, y_sum[:,0], label='RF (mm)',marker='^', color='red',linewidth=1, markersize=6,alpha=0.5)#


ax2.plot(yr,y_sum[:,1], label='In situ (mm)',marker='s', color='green',linewidth=1, markersize=6,alpha=0.5)#
# ax2.plot(yr,rsq_t[:,0], label='rsq (Acre-Feet)',marker='*', color='blue',linewidth=1, markersize=6,alpha=0.5)#


ax2.tick_params(axis='y')
ax1.set_xlim(2005, 2021)
ax1.set_ylim(0, 700)
ax2.set_ylim(0, 100)

# ax2.set_ylim(0, 1)
ax1.tick_params(direction='in')
ax2.tick_params(direction='in')
ax1.legend( loc='upper left')
ax2.legend( loc='upper center')
ax1.xaxis.set_ticks_position('both')

fig.tight_layout()  # otherwise the right y-label is slightly clipped

plt.savefig('C:/Users/swei46.ASURITE/OneDrive/Documents/weekly_repo/manuscript/manuscript_fig/figure_fin_v/figure4_regional_ysum_mm_620.pdf',bbox_inches='tight',dpi=500)
plt.savefig('C:/Users/swei46.ASURITE/OneDrive/Documents/weekly_repo/manuscript/manuscript_fig/figure_fin_v/figure4_regional_ysum__mm_620.png',bbox_inches='tight',dpi=500)
plt.savefig('C:/Users/swei46.ASURITE/OneDrive/Documents/weekly_repo/manuscript/manuscript_fig/figure_fin_v/figure4_regional_ysum__mm_620.tif',bbox_inches='tight',dpi=500)

plt.show()


In [ ]:
np.mean(rsq_t[:,0])


# 4.  yy plot

In [ ]:
'"random split yy plot"'
dat_te=appended_data_te#[(appended_data_te.label>2568)]
dat_tr=appended_data_tr#[(appended_data_tr.label<2568) ]
X_train=dat_tr[variables].to_numpy()
y_train =np.ravel(dat_tr[['irri']]. to_numpy())
X_test=dat_te[variables].to_numpy()
y_test =np.ravel(dat_te[['irri']]. to_numpy())
print(X_test.shape)
print(X_train.shape)
rfm3=RandomForestRegressor(n_jobs=-1,n_estimators= 800,random_state=0,max_features=6,min_samples_leaf=20)
rfm3.fit(X_train,y_train) # fit on train
## predict on val
yhte= rfm3.predict(X_test)
yhtr=rfm3.predict(X_train)
print(rmse(y_test,yhte))
print(rsq(y_test,yhte))
print(rmse(y_train,yhtr))
print(rsq(y_train,yhtr))

In [ ]:
'"regional yy plot"'
dat_te=dat2[(dat2.label>2668)&(dat2.year==6) ]
dat_te_year=dat2[(dat2.label<2668)&(dat2.year==6) ]
dat_tr=dat2[(dat2.label<2668)]
X_train=dat_tr[variables].to_numpy()
y_train =np.ravel(dat_tr[['irri']]. to_numpy())
X_test=dat_te[variables].to_numpy()
y_test =np.ravel(dat_te[['irri']]. to_numpy())
y_train_year=np.ravel(dat_te_year[['irri']]. to_numpy())
rfm = RandomForestRegressor(n_jobs=-1,n_estimators=800,random_state=99,max_features=3,min_samples_leaf=1)
rfm.fit(X_train,y_train)
yhte= rfm.predict(X_test)
yhtr=rfm.predict(X_train)
print(X_test.shape)
print(X_train.shape)
print(rmse(y_test,yhte))
print(rsq(y_test,yhte))
print(rmse(y_train,yhtr))
print(rsq(y_train,yhtr))


In [ ]:
fig, ax = plt.subplots(1,1 , dpi=500)
plt.scatter(y_test,yhte, s=1,marker='o',color='grey',alpha=0.5)
plt.scatter(y_test,y_test,s=0.5,marker='o',color='b',alpha=0.2)
ax.set_xlabel('Insitu Irrigation  (Acre-Feet/grid)')
ax.set_ylabel('RF Predicted Irrigation (Acre-Feet/grid)')
# plt.text(0, 10000, r'$r^2=0.78, RMSE =814$ (Acre-Feet)')#n_tree=8 minleaf=25
# plt.text(0, 7000, '30% Test data')#n_tree=8 minleaf=25
# plt.text(0, 6000, r'$r^2=0.7, RMSE =781$ (Arce_Feet)')
fig.tight_layout() 
# ax.set_xlim(0, 6000)
# ax.set_ylim(0, 6000)
ax.yaxis.set_ticks_position('both')
ax.xaxis.set_ticks_position('both')
ax.tick_params(direction='in')
# plt.grid(color='black', linestyle='-', linewidth=0.5,alpha=0.5)
# ax.axis('square')
# plt.savefig('C:/Users/swei46.ASURITE/OneDrive/Documents/weekly_repo/agu_ppt/random_sp_yyplotwithout_irriarea.pdf',bbox_inches='tight',dpi=500)
# plt.savefig('C:/Users/swei46.ASURITE/OneDrive/Documents/weekly_repo/agu_ppt/random_sp_yyplotwithout_irriarea.png',bbox_inches='tight',dpi=500)

# section 5 pdp

In [ ]:
'" pdp"'
variables_name=["Soybean (%)","Sorghum (%)","Winter Wheat (%)","Corn (%)","Alfalfa (%)","VPD","Precipitation (mm)","Temperature (celsius)","MOD16 ET (mm)","NDVI","GI","NDWI"] 
fig, axes = plt.subplots(nrows=5, ncols=3,figsize=(12, 10),dpi=500,sharey=True)
axes = axes.flatten()
for v in range(len(variables)):
    axes[v]=PartialDependenceDisplay.from_estimator(rfm3, X_train, features=[v], feature_names=variables,ax=axes[v])
fig.tight_layout() 
# axes[12].set_axis_off()
axes[13].set_axis_off()
axes[14].set_axis_off()
# plt.savefig('C:/Users/swei46.ASURITE/OneDrive/Documents/weekly_repo/manuscript/manuscript_fig/PartialDependenceDisplay620_withoutarea.png',bbox_inches='tight',dpi=500)
# plt.savefig('C:/Users/swei46.ASURITE/OneDrive/Documents/weekly_repo/manuscript/manuscript_fig/PartialDependenceDisplay620_withoutarea.pdf',bbox_inches='tight',dpi=500)

# varimport

In [1]:
# imp=np.argsort(rfm3.feature_importances_)[::-1]
# print(rfm3.feature_importances_)
# rfm3.feature_importances_
plt.rcParams.update({
#     "font.weight": "bold",
    "figure.figsize":"15,10",
    "font.size":"30",
#     "grid.color": "0.5",
#     "grid.linestyle": "-",
#     "grid.linewidth": 5,
#     "lines.linewidth": 2,
#     "lines.color": "g",
})
plot_vari_imp=np.mean(vari_imp_mn, axis=0)
variables_name=['Irrigation Area ',"Soybean ","Sorghum ","Winter Wheat ","Corn ","Alfalfa ","VPD","Temperature ","MODIS ET ","Precipitation ","GI","NDVI","NDWI"] 
variables=['denies_area','sb','sgh','ww','corn','alff','vpd','tem','et','pr','gimerged','ndvimerged','ndwimerged'] 

varplot=pd.DataFrame({'group':plot_vari_imp, 'variable':variables_name})
dfx = varplot.sort_values('group',ascending=False)
fig, ax = plt.subplots(1,1 , dpi=500,figsize=(15,10))#,figsize=(15,10)

plt.bar(dfx['variable'],dfx['group'])
ax.set_ylabel('Variable importance',fontsize=40)
ax.xaxis.set_ticks_position('both')
ax.tick_params(direction='in')
ax.yaxis.set_ticks_position('both')
plt.xticks(rotation=90,fontsize=30)

# plt.show()
# fig.savefig('C:/Users/swei46.ASURITE/OneDrive/Documents/weekly_repo//manuscript/manuscript_fig/figure_fin_v/figure2_b_random620_varimp.png',bbox_inches='tight',dpi=500)
# fig.savefig('C:/Users/swei46.ASURITE/OneDrive/Documents/weekly_repo//manuscript/manuscript_fig/figure_fin_v/figure2_b_random620_varimp.pdf',bbox_inches='tight',dpi=500)
# fig.savefig('C:/Users/swei46.ASURITE/OneDrive/Documents/weekly_repo//manuscript/manuscript_fig/figure_fin_v/figure2_b_random620_varimp.tif',bbox_inches='tight',dpi=500)

NameError: name 'plt' is not defined

# intensity of test and train region

In [ ]:
yr=np.arange(2006,2021)
# plt.figure(dpi=500)
fig, ax1 = plt.subplots(dpi=500)

# color = 'tab:red'
ax1.set_xlabel('Year')
ax1.set_ylabel('Growing season (MJJA) sum precipitation (mm)')
ax1.plot(yr, irri_intensity[:,1], label='intensive_train',color='k',alpha=0.3)
ax1.plot(yr, irri_intensity[:,0], label='intensive_test',color='b',alpha=0.3)
ax1.tick_params(axis='y')

# ax2 = ax1.twinx()  # instantiate a second axes that shares the same x-axis

# # color = 'tab:blue'
# # ax2.set_ylabel(r'$R^2$') 
# ax2.set_ylabel('Kansas HPA annual irrigation amount (Acre-Feet)') 

# ax2.plot(yr, y_sum[:,0], label='RF (Acre-Feet)',marker='^', color='red',linewidth=1, markersize=6,alpha=0.5)#
# # ax1.plot(yr, area_sum, label='Irrigation area (Acre-Feet)',marker='^', color='green',linewidth=1, markersize=6,alpha=0.5)#

# ax2.plot(yr, y_sum[:,1], label='In-situ (Acre-Feet)',marker='^', color='green',linewidth=1, markersize=6,alpha=0.5)#
# # ax2.scatter(yr, y_sum[:,0]-y_sum[:,1], label='In-situ (Acre-Feet)')#,marker='^', color='green',linewidth=1, markersize=6,alpha=0.5)#

# # ax2.plot(yr, rsq_t[:,0], label=r'$R^2$', marker='s', color='green',linewidth=1, markersize=6,alpha=0.5)
# ax2.tick_params(axis='y')
# ax1.set_xlim(2005, 2021)
# # ax1.set_ylim(0, 1)
# ax2.set_ylim(0, 1500000)

# ax2.set_ylim(0, 1)
ax1.tick_params(direction='in')
# ax2.tick_params(direction='in')
ax1.legend( loc='upper left')
# ax2.legend( loc='upper right')
ax1.xaxis.set_ticks_position('both')

fig.tight_layout()  # otherwise the right y-label is slightly clipped

# plt.savefig('C:/Users/swei46.ASURITE/OneDrive/Documents/weekly_repo/manuscript/manuscript_fig/y_sum_regional_withprecipitation.pdf',bbox_inches='tight',dpi=500)
# plt.savefig('C:/Users/swei46.ASURITE/OneDrive/Documents/weekly_repo/manuscript/manuscript_fig/y_sum_regional_withprecipitation.png',bbox_inches='tight',dpi=500)
# # plt.show()


# spatial plot

In [ ]:
'"regional yy plot"'
yr=6
dat_te=dat2[(dat2.label>2668)&(dat2.year==yr) ]
dat_te_year=dat2[(dat2.label<2668)&(dat2.year==yr) ]
dat_tr=dat2[(dat2.label<2668)]
X_train=dat_tr[variables].to_numpy()
y_train =np.ravel(dat_tr[['irri']]. to_numpy())
X_test=dat_te[variables].to_numpy()
y_test =np.ravel(dat_te[['irri']]. to_numpy())
y_train_year=np.ravel(dat_te_year[['irri']]. to_numpy())
rfm = RandomForestRegressor(n_jobs=-1,n_estimators=800,random_state=seed_no,max_features=3,min_samples_leaf=20)
rfm.fit(X_train,y_train)
yhte= rfm.predict(X_test)
yhtr=rfm.predict(X_train)
print(X_test.shape)
print(X_train.shape)
print(rmse(y_test,yhte))
print(rsq(y_test,yhte))
print(rmse(y_train,yhtr))
print(rsq(y_train,yhtr))

In [ ]:
def reshape_rf_intensity(dat2,yhte,yr):
    dat_te_year=dat2[(dat2.label<2668)&(dat2.year==yr) ]
    dat_te=dat2[(dat2.label>2668)&(dat2.year==yr) ]
    "'add label to test result '"
    irri_int=np.array([yhte/dat_te['denies_area']])
    y_te_new=irri_int.reshape(len(yhte),1)
    dat_te_new=dat_te.label.to_numpy().reshape(len(dat_te.label),1)
    new_te=np.append(y_te_new,dat_te_new,axis=1)
    "'add label to train result '"
    y_train_year=np.ravel(dat_te_year['irri']/dat_te_year['denies_area']. to_numpy())
    y_train_year_new=y_train_year.reshape(len(y_train_year),1)
    dat_tr_year_new=dat_te_year.label.to_numpy().reshape(len(dat_te_year.label),1)
    new_tr=np.append(y_train_year_new,dat_tr_year_new,axis=1)
    "'append tr and test irri'"
    tmp_dat=np.append(new_te,new_tr,axis=0)
    df = pd.DataFrame(tmp_dat,columns=['irri', 'grid']).set_index('grid')
    kk=df.sort_index(axis = 0) # sort
    grid=pd.DataFrame(np.arange(1,8063),columns=['grid'])# full grid
    tmp_cat=pd.concat([kk, grid], axis=1).set_index('grid')# fill in irri info to grid
    fin_map=tmp_cat.to_numpy().reshape(139,58)# reshape final array
    intensity=fin_map.T# transpose
    return (intensity)

In [ ]:
def reshape_rf_irri(dat2,yhte,yr):
    dat_te_year=dat2[(dat2.label<2668)&(dat2.year==yr) ]
    dat_te=dat2[(dat2.label>2668)&(dat2.year==yr) ]
    "'add label to test result '"
    y_te_new=yhte.reshape(len(yhte),1)
    dat_te_new=dat_te.label.to_numpy().reshape(len(dat_te.label),1)
    new_te=np.append(y_te_new,dat_te_new,axis=1)
    "'add label to train result '"
    y_train_year=np.ravel(dat_te_year['irri']. to_numpy())
    y_train_year_new=y_train_year.reshape(len(y_train_year),1)
    dat_tr_year_new=dat_te_year.label.to_numpy().reshape(len(dat_te_year.label),1)
    new_tr=np.append(y_train_year_new,dat_tr_year_new,axis=1)
    "'append tr and test irri'"
    tmp_dat=np.append(new_te,new_tr,axis=0)
    df = pd.DataFrame(tmp_dat,columns=['irri', 'grid']).set_index('grid')
    kk=df.sort_index(axis = 0) # sort
    grid=pd.DataFrame(np.arange(1,8063),columns=['grid'])# full grid
    tmp_cat=pd.concat([kk, grid], axis=1).set_index('grid')# fill in irri info to grid
    fin_map=tmp_cat.to_numpy().reshape(139,58)# reshape final array
    rf_map=fin_map.T# transpose
    return (rf_map)

In [ ]:
def reshape_insitu_irri (dat2, yr):
    dat=dat2[(dat2.year==yr) ]
    index=['irri','label']
    tr_dat=dat[index].set_index('label')
    tr_dat.index.names = ['grid']
    grid=pd.DataFrame(np.arange(1,8063),columns=['grid'])
    tmp_cat=pd.concat([tr_dat,grid],axis=1).set_index('grid')
    tr_d=tmp_cat.to_numpy().reshape(139,58)
    tr_map=tr_d.T
#     tr_map[tr_map == 0] = 'nan'
    return (tr_map)

In [ ]:
rf_map=reshape_rf_irri (dat2,yhte,yr)

In [ ]:
tr_map=reshape_insitu_irri (dat2, yr)

In [ ]:
intensity=reshape_rf_intensity(dat2,yhte,yr)

In [ ]:
lon_start = -102.1;
lon_end = -94.6;
lat_start = 36.96; 
lat_end = 40.05;
per_cell= 0.054;
lon_step=np.arange(lon_start,lon_end,per_cell)
lat_step=np.arange(lat_start,lat_end,per_cell)
lon_ks,lat_ks=np.meshgrid(lon_step,lat_step) 

In [ ]:

np.savetxt('D:/all_GEE/lat_ks.csv',lat_ks,delimiter=',')
np.savetxt('D:/all_GEE/lon_ks.csv',lon_ks,delimiter=',')

In [ ]:

np.savetxt('D:/all_GEE/rf_map' +str(yr)+'.csv',rf_map,delimiter=',')
np.savetxt('D:/all_GEE/tr_map' +str(yr)+ '.csv',tr_map,delimiter=',')
np.savetxt('D:/all_GEE/intensity'+str(yr)+ '.csv',intensity,delimiter=',')

In [ ]:
# levels = np.linspace(0, 1, 21)

plt.contourf(lon_ks,lat_ks,rf_map, cmap='rainbow')

plt.xlim(-102, -97)
plt.xlabel('longitude')
plt.ylabel('latitude')
cbar=plt.colorbar()
cbar.set_label('Acre-Feet', rotation=270)
plt.show()

In [ ]:
dat=dat2[(dat2.year==yr) ]
index=['irri','label']
tr_dat=dat[index].set_index('label')
tr_dat.index.names = ['grid']
grid=pd.DataFrame(np.arange(1,8063),columns=['grid'])#
tmp_cat=pd.concat([tr_dat,grid],axis=1).set_index('grid')
tr_d=tmp_cat.to_numpy().reshape(139,58)
tr_map=tr_d.T

In [ ]:
tmp_cat.iloc[5062]

# shape file show.0

In [ ]:
import shapefile
from shapely.geometry import Polygon, Point
import os
os.environ["PROJ_LIB"] = os.path.join(os.environ["CONDA_PREFIX"], "share", "proj")
from mpl_toolkits.basemap import Basemap